**Aeronautics Institute of Technology – ITA**

**Computer Vision – CM-203**

**Professors:** 

Marcos Ricardo Omena de Albuquerque Maximo

Gabriel Adriano de Melo


**Instructions:**

Before submitting your lab, be sure that everything is running correctly (in sequence): first, **restart the kernel** (`Runtime->Restart Runtime` in Colab or `Kernel->Restart` in Jupyter). Then, execute all cells (`Runtime->Run All` in Colab or `Cell->Run All` in Jupyter) and verifies that all cells run without any errors, expecially the automatic grading ones, i.e. the ones with `assert`s.

**Do not delete the answer cells**, i.e. the ones that contains `WRITE YOUR CODE HERE` or `WRITE YOUR ANSWER HERE`, because they contain metadata with the ids of the cells for the grading system. For the same reason, **do not delete the test cells**, i.e. the ones with `assert`s. The autograding system executes all the code sequentially, adding extra tests in the test cells. There is no problem in creating new cells, as long as you do not delete answer or test cells. Moreover, keep your solutions within the reserved spaces.

The notebooks are implemented to be compatible with Google Colab, and they install the dependencies and download the datasets automatically. The commands which start with ! (exclamation mark) are bash commands and can be executed in a Linux terminal.

---

# Camera Model Laboratory

In this laboratory, we will implement a camera model and use some OpenCV functions for Perspective-n-Point, camera calibration, and stereo vision.

This lab was originally created by Gabriel Melo. Marcos Maximo translated it to English and made some minor improvements.

In [ ]:
# !pip3 install opencv-contrib-python==4.8.0.74 Pillow==9.4.0 matplotlib==3.7.1 scipy==1.11.3 gdown==4.6.0 numpy==1.23.5

import cv2
import numpy as np
import PIL.Image
from matplotlib import pyplot as plt
from glob import glob
from pathlib import Path
from scipy.spatial.transform import Rotation as R

plt.style.use('seaborn-darkgrid')

If you are going to execute this locally and want to change the Path, you can do it without any problems for the automatic grading.

In [ ]:
! [ ! -d "/content/calib_esq" ] && gdown -O /content/calib_esq.zip 1vg2fnoLjcYAdF44HxwK41basZUj0xYMN && unzip /content/calib_esq.zip -d /content && rm /content/calib_esq.zip
! [ ! -d "/content/calib_dir" ] && gdown -O /content/calib_dir.zip 1d0OeP9YCxx1sWjwDMvf0y6f35Lx3ELPN && unzip /content/calib_dir.zip -d /content && rm /content/calib_dir.zip
! [ ! -d "/content/tsukuba" ] && gdown -O /content/tsukuba.zip 1Ghpx9_x8E26SzJP3X-Itt94QD3yVM8lT && unzip /content/tsukuba.zip -d /content && rm /content/tsukuba.zip
root_path = Path("/content")
imgs_left_path = root_path/"calib_esq"
imgs_right_path = root_path/"calib_dir"
tsukuba_path = root_path/"tsukuba"

Tip: if you want to access the documentation of a given function, add a question mark to the end of the function name. Example:

In [ ]:
cv2.imread?

## Camera Model

Initially, you will implement some equations related to the camera model we presented in class.

### Equivalence Between Convergent Lens and Pinhole

Firstly, we will derive some equations regarding the camera model. In our perspective projection model, the light rays converge into a single point (optical center) and form an inverted image in a plane (sensor plane).
The thin lens camera is equivalent to the pinhole model.


**OBSERVATION:** As explained in class, the focal lengths of the pinhole model and of a covergent lens are not the same. However, they became basically the same when the object is far from the camera. When we usually focal length in computer vision, we mean the focal length of the pinhole model $f_{pinhole}$, which is the distance between the projection center and the projection plane, as illustrated in the figure below.

![Modelo de Câmera Pinhole](https://gam.dev/cm203/imgs/pinhole.svg)

The lens (or system of lens) has its own focal length $f_{lens}$. Moreover, we also have the distance from the lens to the image $d_i$. The relationship between the elements in the figure below is given by the "Thin Lens Equation".

Therefore, we basically have an equivalence between $d_i$ and $f_{pinhole}$. When the object is far from the lens, $d_o >> f_{lens}$, then $d_i \approx f_{lens}$.

![Modelo de Câmera com Lente Equivalente](https://gam.dev/cm203/imgs/lens.svg)

Let us compute how $f_{pinhole}$ relates to $f_{lens}$ and $d_o$ using the "Thin Lens Equation".

To express your solution, implement the function below (0.5 points)

In [ ]:
def focal_length_pinhole(f_lens: np.ndarray, d_o: np.ndarray) -> np.ndarray:
    """
    Implements the equation that computes the focal length equivalent to a pinhole camera (d_i).
    To make testing easier, the implementation considers that the inputs and the output are NumPy arrays.
    Therefore, use element-wise operations for these arrays.
    :param f_lens: the focal length of a convergent lens, in meters.
    :param d_o: distance from the lens to the object plane in focus, in meters.
    :return: the focal length equivalent to a pinhole camera, in meters.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return f_pinhole

Notice that a test for a focal length of 8 mm and a focal plane of 10 m, $f_{pinhole}$ is basically equal to $f_{lens}$, with a difference smaller than 0.08%.

In [ ]:
assert abs(focal_length_pinhole(0.008,  10) - 0.00800640) < 1e-6
assert abs(focal_length_pinhole(np.array([0.008]),  np.array([10.])) - 0.00800640) < 1e-6

### Field of View (FoV)

Now, let us compute the FoV (in degrees) for a pinhole camera taking into account the sensor width and the distance from the sensor to the projection center.

![Campo de Visão (Field of View) FoV](https://gam.dev/cm203/imgs/fov.svg)

Implement the function below to compute the FoV (0.5 points).

**Tip:**
Use the function `np.arctan` and the constant `np.pi`.

In [ ]:
def compute_fov(f_pinhole: np.ndarray, sensor_width: np.ndarray) -> np.ndarray:
    """
    Computes the field of view in degrees for a pinhole camera.
    Consider that the inputs and output are NumPy arrays and use element-wise operations.
    :param f_pinhole: focal length, in meters.
    :param sensor_width: sensor width, in meters.
    :return: field of view in degrees.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return fov

In [ ]:
assert abs(compute_fov(1,  2) - 90) < 1e-6
assert abs(compute_fov(np.array([1.]),  np.array([2.]))[0] - 90) < 1e-6

Let us do the same computation, but taking into account a thin lens camera focused on a object plane at a distance $d_o$ (0.5 points).

**Tip:**
Use the relationship between $f_{pinhole}$ and $f_{lens}$ you computed previously.

In [ ]:
def compute_fov_lens(f_lens: np.ndarray, sensor_width: np.ndarray, d_o: np.ndarray) -> np.ndarray:
    """
    Computes the field of view in degrees for a thin lens camera.
    Consider that the inputs and output are NumPy arrays and use element-wise operations.
    :param f_lens: focal length of the lens, in meters.
    :param sensor_width: sensor width, in meters.
    :return: the field of view in degrees.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return fov

In [ ]:
assert abs(compute_fov_lens(0.001,  0.002, 100) - 89.999427) < 1e-6
assert abs(compute_fov_lens(np.array([0.001]),  np.array([0.002]), np.array([100.]))[0] - 89.999427) < 1e-6

Notice that the FoV changes very little with respect to the distance to the plane in focus. The change is larger for objects which are very close.

In [ ]:
plt.plot(np.linspace(0.07, 5, 99), compute_fov_lens(np.array([0.001]*99), np.array([0.002]*99), np.linspace(0.07, 5,99)))
plt.xlabel('Plane in focus (m)')
plt.ylabel('Plane in focus (degrees)')
plt.show()

Now, let's consider a more practical setting, where the camera manufacturer gives us the field of view in degrees and we want to be able to compute the perspective projection for this camera. Therefore, let's compute $f_{pinhole}$ so we can construct the intrinsic matrix related to this camera. We will compute the horizontal FoV, but the calculation is the same for the vertical axis (0.5 points).

**Tip:**
Use `np.tan`.

In [ ]:
def compute_focal_length(fov: float, sensor_width: float) -> float:
    """
    Computes the focal length in pixels for a pinhole camera.
    :param fov: field of view in degrees.
    :param sensor_width: sensor width in pixels.
    :return: the focal length in pixels.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return f_pinhole

In [ ]:
assert abs(compute_focal_length(90, 1920) - 960) < 1e-6

In the exercise above, the focal length is in pixels, but if we want to convert it to meters, we will need to know the pixel size, or equivalently, the sensor width. Then, we can just make a unit conversion from pixels to meters.

### Homogeneous Coordinates and Projective Geometry

Let's go now to the projective model:

To simplify, we consider the projection plane in front of the projection center.

Notice that the result is equivalent: the projected image has exactly the same dimensions, and the focal length is the same in absolute value.

![Câmera Pinhole Plano de projeção Frontal](https://gam.dev/cm203/imgs/pinhole_cam.svg)

Pay attention to our camera coordinate system, where our origin is the Projection Center. Our projection plane is orthogonal to the Z axis, which points outward of the camera.

![Câmera Pinhole Plano de projeção Frontal](https://gam.dev/cm203/imgs/pinhole_cam_proj.svg)


A point in the real world $(X, Y, Z)$ measured in meters is projected to the image plane as: $(X_p, Y_p, Z_p) = (X\frac{f_{metros}}{Z}, Y\frac{f_{metros}}{Z}, f)$.


![Câmera Pinhole Plano de projeção Frontal](https://gam.dev/cm203/imgs/pinhole_cam_proj_ext.svg)


Since we have a division by $Z$, this is not a linear transform and we can't write it in matricial form. To solve this issue, we introduce the concept of homogenous coordinates.

To transform a $N$-dimensional Cartesian point in its homogenous equivalent, we only need to include a new dimension with unit value. Inversely, we need to divide by this value, taking care with points that go to infinite (division by zero). Therefore, the 2D Cartesian point $(X, Y)$ becomes $(X, Y, 1)$ in homogeneous coordinates. Moreover, the homogeneous point $(X, Y, Z, W)$ becomes $(X/W, Y/W, Z/W)$ in the Cartesian space. Notice that infinite homogeneous points are mapped into the same Cartesian point.

Therefore, in the exercises below (0.5 points each), implement the transform from Cartesian coordinates to homogeneous coordinates and vice-versa. Don't worry about points in the infinite.

**Tip:**
Use `np.vstack` (which receives a tuple of column vectors) to concatenate two arrays vertically.

In [ ]:
def cartesian_to_homogeneous(cartesian: np.ndarray) -> np.ndarray:
    """
    Transforms from Cartesian to homogeneous coordinates.
    :param cartesian: column array in Cartesian coordinates.
    :return: column array in homogeneous coordinates.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return homogeneous

In [ ]:
assert np.linalg.norm(cartesian_to_homogeneous(np.array([[2],
                                                          [4]])) - np.array([[2],
                                                                             [4],
                                                                             [1]])) < 1e-6
assert np.linalg.norm(cartesian_to_homogeneous(np.array([[19],
                                                          [19],
                                                          [19]])) - np.array([[19],
                                                                              [19],
                                                                              [19],
                                                                              [1 ]])) < 1e-6

In [ ]:
def homogeneous_to_cartesian(homogeneous: np.ndarray) -> np.ndarray:
    """
    Transforms from homogeneous to Cartesian coordinates.
    This function doesn't take into account points in infinite.
    :param homogeneous: column array in homogeneous coordinates.
    :return: column array in Cartesian coordinates.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return cartesian

In [ ]:
assert np.linalg.norm(homogeneous_to_cartesian(np.array([[4],
                                                          [8],
                                                          [2]])) - np.array([[2],
                                                                             [4]])) < 1e-6
assert np.linalg.norm(homogeneous_to_cartesian(np.array([[19],
                                                          [19],
                                                          [19],
                                                          [19]])) - np.array([[1],
                                                                              [1],
                                                                              [1]])) < 1e-6

However, we don't want to know where the point is in meters, we want to know where the point in is the image in pixels. Therefore, let's use a simple rule of three (linear transform):

We first need to know where the point $(0, 0, f_{meters})$ of the projection plane is in the image in pixels. We define this position as the center of the image: $(c_x, c_y)$. Then, we need to know the relationship between pixels and meters to transform the focal length in meters to pixels, i.e., we need the pixel size $\mu$.

Therefore, our 3D point $(X_p, Y_p, Z_p)$ measured in meters becomes a 2D point $\left( c_x + X_p \mu, c_y + Y_p \mu \right)$ measured in pixels. Substituting, we have $\left(c_x + X\frac{f_{metros}}{Z}\mu, c_y + Y\frac{f_{metros}}{Z}\mu\right)$.

Notice that the term $\mu f_{metros}$ is the focal length in pixels, which we'll call now simply as $f$.

Therefore, we have a mapping from a 3D point $(X, Y, Z)$ in meters to $\left(c_x + X\frac{f}{Z}, c_y + Y\frac{f}{Z}\right)$, where the values $c_x$, $c_y$, and $f$ are measured in pixels.

Implement the perspective projection below (0.5 pontos). The output is given by $(f X + c_x Z, f Y + c_y Z, Z)$.

**Tip:**
Use the operator `@` from NumPy for matrix multiplication `A @ B`, which is equivalent to `np.dot(A, B)`.

In [ ]:
def projective_transform(point3d: np.ndarray, c_x: float, c_y: float, f: float) -> np.ndarray:
    """
    Projects a point from space to a point in the image plane represented with homogeneous coordinates.
    :param c_x: x coordinate of the image center, in pixels.
    :param c_y: y coordinate of the image center, in pixels.
    :param f: focal length in pixels.
    :return: a column array in homogeneous coordinates and the transform matrix (intrinsic matrix).
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return homogeneous_point, intrinsic_matrix

In [ ]:

point, mtx = projective_transform(np.array([[1],[1],[1]]), 1, 1, 1) # Tudo 1 para não espiar :)
assert np.linalg.norm(mtx/mtx[-1,-1] - np.array([[1, 0, 1], [0, 1, 1], [0, 0, 1]])) < 1e-6
assert np.linalg.norm(homogeneous_to_cartesian(point) - np.array([[2], [2]])) < 1e-6

Beyond the previous mapping which takes to $\left(c_x + X\frac{f}{Z}, c_y + Y\frac{f}{Z}\right)$, we can also consider a focal length different for each axes $x$ and $y$. This may happen in practice if the pixel sensor is not squared. In this setting, we call *aspect ratio* $f_y = a f_x$.

Another effect is the image *skew*, which can model the case where the sensor lines are misaligned, making a parallelogram instead of a rectangle.

Therefore, a more complete model would be: $\left(c_x + \frac{X f_x + Y skew}{Z}, c_y + Y\frac{f_y}{Z}\right)$. In the function below, define a new intrinsic matrix which takes into account the effects discussed here (0.5 points).

In [ ]:
def intrinsic_matrix_skew(c_x: float, c_y: float, f_x: float, f_y: float, skew: float) -> np.ndarray:
    """
    Builds a intrinsic matrix for a camera defined by the parameters c_x, c_y, f_x, f_y, skew.
    Consider that the output is in homogeneous coordinates.
    :param c_x: x coordinate of the image center, em pixels.
    :param c_y: y coordinate of the image center, em pixels.
    :param f_x: focal length in pixels for the axis x.
    :param f_y: focal length in pixels for the axis y.
    :param skew: skew coefficient, in pixels.
    :return: the intrinsic matrix which considers that f_x != f_y and the existence of a skew coefficient.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return intrinsic_matrix

In [ ]:
mtx = intrinsic_matrix_skew(1, 1, 1, 1, 1) # Tudo 1 para não espiar :) mas temos mais testes, cuida
assert np.linalg.norm(mtx - np.array([[1, 1, 1], [0, 1, 1], [0, 0, 1]])) < 1e-6

## Perspective-n-Point

Let's do a direct application of Perspective-n-Point (PnP). The PnP algorithm finds the pose (rotation and translation) which minimize the reprojection error, given the intrinsic parameters of the camera (which are known).

Find the pose of the camera using the function `cv2.solvePnP`. To understand how to use this function, see its documentation in the next cell. 
Moreover, use the flag `cv2.SOLVEPNP_EPNP`, because the default one, which is `cv2.SOLVEPNP_ITERATIVE`, requires more points if a initial guess for `[R, T]` is not given (1 point).

In [ ]:
cv2.solvePnP?

In [ ]:
def pnp(verts3d, verts2d, camera_params):
    """
    Obtaints the pose of an object in space, given its geometry, the corresponding 2D points, and
    the intrinsic parameters of the camera.
    :param verts3d: the coordinates of the 3D points of the object in its own coordinate system, in meters.
    :param verts2d: the corresponding 2D points in the image, in pixels.
    :param camera_parameters: tuple with the intrinsic matrix and the distortion coefficients.
    :return: the reprojection error and the rotation (in Rodrigues notation, in radians) and translation vectors.
    """
    intrinsic_matrix, distortion_coeffs = camera_params
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return reproj_error, rvec, tvec

Notice that in the test below, we start from random pose and geometry.

In [ ]:
verts3d = np.array([[-1,-1,0],[-1,1,0],[1,1,0],[1,-1,0],[0,0,2]], dtype=np.float32)
mtx = np.array([[1000,0,1000],[0,1000,500],[0,0,1]], dtype=np.float32)
dist = np.zeros((5,), dtype=np.float32)
rvec = np.array([[0.3], [-0.2], [0.6]])
tvec = np.array([[0.1], [-0.2], [7]])
verts2d, _ = cv2.projectPoints(verts3d, rvec, tvec, mtx, dist)
reproj_error, rvec_rec, tvec_rec = pnp(verts3d, verts2d, (mtx, dist))
assert np.linalg.norm(R.from_rotvec(rvec.ravel()).as_matrix() - R.from_rotvec(rvec_rec.ravel()).as_matrix()) < 0.01
assert np.linalg.norm(tvec - tvec_rec) < 0.001

Let's visualize the projection of these points in space. Feel free to change the values of rvec, tvec, and the intrinsic matrix.

We are only connecting the projected points with green lines so we can have an idea of the object in space.

In [ ]:
img = np.zeros((1000, 2000, 3), dtype=np.uint8)
f = 1000 * 1
d = 4 * 1
mtx = np.array([[f,0,1000],[0,f,500],[0,0,1]], dtype=np.float32)
rvec = np.array([[0.7], [0.3], [0]], dtype=np.float32)
tvec = np.array([[0.1], [0.8], [d]])
projected_points, _ = cv2.projectPoints(verts3d, rvec, tvec, mtx, dist)
for i, p1 in enumerate(projected_points):
    if i + 2 < len(projected_points):
        p2 = projected_points[i+1].astype(np.uint).ravel()
    else:
        p2 = projected_points[0].astype(np.uint).ravel()
    cv2.line(img, p1.astype(np.uint).ravel(), p2, (0, 255, 0))
    cv2.line(img, p1.astype(np.uint).ravel(), projected_points[-1].astype(np.uint).ravel(), (0, 255, 0))
    cv2.circle(img, p1.astype(np.uint).ravel(), 4, (0, 255, 0))
cv2.drawFrameAxes(img, mtx, dist, rvec, tvec, 1)

PIL.Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

## Stereo Calibration

In stereo calibration, we wan to find the camera pose with respect to another camera. In this case, we can use the intrinsic parameters already estimated in the monocular calibration (but they can also be refined if needed).

In [ ]:
def get_verts2d(path, nverts, criteria):
    pathnames = [str(p) for p in sorted(path.glob('*.png'))]
    imgs = [cv2.cvtColor(cv2.imread(p)[:,:,0], cv2.COLOR_BAYER_GB2GRAY) for p in pathnames]
    verts2d = [cv2.findChessboardCorners(img, nverts) for img in imgs]
    return [cv2.cornerSubPix(img, v, (7,7), (-1,-1), criteria) if r else None for img, (r, v) in zip(imgs, verts2d)]

image_size = cv2.imread(str(imgs_left_path/'00.png')).shape[1::-1]
nverts = (10, 7)

l_square = 0.1 # metros
verts3d = np.zeros((nverts[0] * nverts[1], 3), dtype=np.float32)
i = 0
for y in range(nverts[1]):
    for x in range(nverts[0]):
        verts3d[i] = (x * l_square, y * l_square, 0)
        i += 1

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 5e-4)
verts2d_left = get_verts2d(imgs_left_path, nverts, criteria)
verts2d_right = get_verts2d(imgs_right_path, nverts, criteria)

You'll now do a stereo calibration to find the rotation and translation from one camera with respect to the other one. To do this, use the function `cv2.stereoCalibrateExtended`. Notice that the input `objectPoints` of the OpenCV function is a list of arrays (representing a set of the 3D points) for each image, while `verts3d` in our function is a single array. Moreover, take into account that we may not detect points in some images, so you may need to filter some elements of `verts2d` to keep only situations where corresponding points are found on the left and right images.
The stereo calibration is also able to estimate intrinsic parameters, but it is usually better to find these parameters using a monocular calibration. The default flag cv2.CALIB_FIX_INTRINSIC already considers fixed intrinsic parameters during the calibration. For the inputs R and T, we can use `None` or give null vectors (1.5 points).

In [ ]:
cv2.stereoCalibrateExtended?

In [ ]:
def stereo_calibration(verts3d, verts2d_left, verts2d_right, cam_left, cam_right, image_size):
    """
    Executes a stereo calibration for a pair of cameras, without optimizing the intrinsic parameters.
    :param verts3d: the coordinates of the 3D points in its own coordinate system.
    :param verts2d_left: list of 2D points found in the images captured by the left camera.
    :param verts2d_right:  list of 2D points found in the images captured by the right camera.
    :param cam_left: tuple with the intrinsic matrix and distortion coefficients of the left camera.
    :param cam_right: tuple with the intrinsic matrix and distortion coefficients of the right camera.
    :param image_size: tuple with the image size in pixels.
    :return: the mean reprojection error of the calibration, the rotation matrix and the translation vector that
    takes the 3D points from the left camera's coordinate system to the right one, the essential matrix, the
    fundamental matrix, and the reprojection error for each image.
    """
    ml, dl = cam_left
    mr, dr = cam_right
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return mean_error, rotation, translation, essential, fundamental, per_view_error

In [ ]:
cam_left = (np.array([[1.27677228e+03, 0.00000000e+00, 9.46863329e+02],
                     [0.00000000e+00, 1.28093814e+03, 5.26800967e+02],
                     [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]], dtype=np.float32),
    np.array([ 0.00463763,  0.00506855, -0.00252541, -0.00054011,  0.        ], dtype=np.float32))
cam_right = (np.array([[1.28316167e+03, 0.00000000e+00, 9.83705999e+02],
                     [0.00000000e+00, 1.28959033e+03, 4.74866554e+02],
                     [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
    np.array([ 0.00438509,  0.01102175, -0.00740251,  0.00029485,  0.        ], dtype=np.float32))
err, rmat, T, E, F, per_view_error = stereo_calibration(verts3d, verts2d_left[:-6], 
                                                        verts2d_right[:-6], cam_left, cam_right, image_size)
assert err < 0.7
assert np.mean(per_view_error) < 0.7
assert per_view_error.shape == (28, 2)
assert np.linalg.norm(rmat - np.array([[ 0.9330993 , -0.00167611,  0.35961491],
                                       [-0.00388421,  0.99988383,  0.01473875],
                                       [-0.35959784, -0.01514954,  0.9329844 ]])) < 1e-5
assert np.linalg.norm(T - np.array([[-9.54630632e-01],[ 3.48152961e-04],[ 1.50440555e-01]])) < 1e-5
assert np.linalg.norm(E - np.array([[ 4.59148223e-04, -1.50428353e-01, -1.89248456e-03],
                                    [-2.02907133e-01, -1.47143694e-02,  9.44756152e-01],
                                    [ 3.38312816e-03, -9.54519153e-01, -1.41952640e-02]])) < 1e-5
assert np.linalg.norm(F - np.array([[-3.71746733e-09,  1.21397368e-06, -6.16439367e-04],
                                    [ 1.63463644e-06,  1.18154655e-07, -1.13275774e-02],
                                    [-8.07724752e-04,  8.63399180e-03,  1.00000000e+00]])) < 1e-5

## Stereo Retification

Find homographies for each one of the cameras so we have parallel epipolar lines (epipoles at the infinity).

Use the function `cv2.stereoRectify` to find the retification matrix for each one of the cameras. Beyond this homography, it also updates the projection matrix (i.e. the matrix that results from the multiplication between intrinsic and extrinsic matrices `K[RT]`) (1.5 points).

In [ ]:
cv2.stereoRectify?

In [ ]:
def stereo_retification(cam_left, cam_right, image_size, rotation, translation):
    """
    Executes the stereo retification to make the epipolar lines parallel to the projection of the horizontal lines.
    :param cam_left: tuple with the intrinsic matrix and the distortion coefficients of the left camera.
    :param cam_right: tuple with the intrinsic matrix and distortion coefficients of the right camera.
    :param image_size: tuple with the image size in pixels.
    :param rotation: rotation matrix of the left camera measured in the coordinate system of right one.
    :param translation: left camera's position measured in the coordinate system of right one.
    :return: the left and right retification matrices, the left and right projection matrices, the depth-to-disparity
             mapping matrix.
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()
    return retification_left, retification_right, projection_left, projection_right, Q

def apply_mapping(image, camera_params, retification_matrix, projection_matrix, image_size):
    mappings = cv2.initUndistortRectifyMap(*camera_params, retification_matrix, projection_matrix,
                                           image_size, cv2.CV_32FC1)
    return cv2.remap(image, *mappings, cv2.INTER_LANCZOS4)

In [ ]:
R1, R2, P1, P2, Q = stereo_retification(cam_left, cam_right, image_size, rmat, T)
assert np.linalg.norm(R1 - np.array([[ 9.77703755e-01,  3.42425370e-04,  2.09988690e-01],
                                     [-1.94226256e-03,  9.99970641e-01,  7.41250433e-03],
                                     [-2.09979987e-01, -7.65508649e-03,  9.77675715e-01]])) < 1e-5
assert np.linalg.norm(P1 - np.array([[1.28526422e+03, 0.00000000e+00, 9.02986206e+02, 0.00000000e+00],
                                     [0.00000000e+00, 1.28526422e+03, 4.93165016e+02, 0.00000000e+00],
                                     [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])) < 1e-5

Observe the images after retification.

In [ ]:
img_left = cv2.cvtColor(cv2.imread(str(imgs_left_path/'23.png'), cv2.IMREAD_ANYCOLOR), cv2.COLOR_BAYER_GB2RGB)
img_right = cv2.cvtColor(cv2.imread(str(imgs_right_path/'23.png'), cv2.IMREAD_ANYCOLOR), cv2.COLOR_BAYER_GB2RGB)

img_left_ret_full = apply_mapping(img_left, cam_left, R1, P1, image_size)
img_right_ret_full = apply_mapping(img_right, cam_right, R2, P2, image_size)
img_left_ret_full = np.clip(img_left_ret_full.astype(np.float32)*1.3, 0, 255).astype(np.uint8)
img_right_ret_full = np.clip(img_right_ret_full.astype(np.float32)*1.3, 0, 255).astype(np.uint8)

In [ ]:
PIL.Image.fromarray(img_left_ret_full)

In [ ]:
PIL.Image.fromarray(img_right_ret_full)

## Disparity

Let's now use a correlator that uses both retified images, i.e. it supposes that the epipolar constraint happens only in the horizontal lines (horizontal and parallel epipolar lines).

Feel free to change the parameters below, especially the values of `blockSize` which defines the size of the correlation window and `speckleWindowSize` and `speckleRange` which filter spurious points. Notice that the cameras are separated by 1 meter (and they were also inclined), the disparity gets to the order of 500 pixels for the closest point, which can be seen by `numDisparities=16*32` = 512 (160 after the resize). Only for points at the infinity we'll have `minDisparity=0`.

Notice that due to the lack of texture in the image, especially on the ground and on the wall, most points don't have a defined depth, because the algorithm couldn't find its match. Since the disparity are also large the algorithm finds many spurious correlations.

Moreover, we also need to reduce the image size to have less noisy results. Try to experiment with other values of `s`, which incrase or reduce the scale.

In [ ]:
s = 0.22
img_left_ret = cv2.resize(img_left_ret_full, None, None, s, s, cv2.INTER_AREA)
img_right_ret = cv2.resize(img_right_ret_full, None, None, s, s, cv2.INTER_AREA)
print(img_left_ret.shape)
PIL.Image.fromarray(img_left_ret)

In [ ]:
correlator = cv2.StereoSGBM_create(minDisparity=0, numDisparities=int(16*32*s), blockSize=11, 
                  P1=8*3*11*11, P2=32*3*11*11, speckleWindowSize = 50, speckleRange = 2, disp12MaxDiff=20)
corr_left = correlator.compute(img_left_ret, img_right_ret)

plt.figure(figsize=(16,9))
plt.axis(False)
plt.imshow(corr_left, cmap='gray', vmin=0)
plt.plot()

We can reapply the correlator with the images switched and later use a filter between the antipode results.

Notice that the colors are inverted, in reality this happens because the disparity that was previously positive is now negative.

In [ ]:
inverse_correlator = cv2.ximgproc.createRightMatcher(correlator)
corr_right = inverse_correlator.compute(img_right_ret, img_left_ret)

plt.figure(figsize=(16,9))
plt.axis(False)
plt.imshow(corr_right, cmap='gray', vmax=0)
plt.plot()

Applying the filter:

In [ ]:
filt = cv2.ximgproc.createDisparityWLSFilter(matcher_left=correlator)
filt.setLambda(1300)
filt.setSigmaColor(1.9)
filtered_result = filt.filter(np.int16(corr_left), img_left_ret, None, np.int16(corr_right))

plt.figure(figsize=(16,9))
plt.axis(False)
plt.imshow(filtered_result, cmap='gray',vmin=-16*16*2)
plt.plot()

For an image with more texture and smaller disparities (in the order of 16 pixels):

(Obs.: the image is already retified)

In [ ]:
ileft = cv2.imread(str(tsukuba_path/"tsukuba_l.png"), cv2.IMREAD_ANYCOLOR)
iright = cv2.imread(str(tsukuba_path/"tsukuba_r.png"), cv2.IMREAD_ANYCOLOR)

In [ ]:
correlator = cv2.StereoSGBM_create(minDisparity=-16, numDisparities=32, blockSize=9, 
                  P1=4*3*9*9, P2=16*3*9*9, speckleWindowSize = 50, speckleRange = 1, disp12MaxDiff=20)

corr_left = correlator.compute(ileft, iright)


plt.figure(figsize=(16,9))
plt.axis(False)
plt.imshow(corr_left, cmap='gray', vmin=0)
plt.plot()

# Your data and feedback:

Write a feedback for the lab so we can make it better for the next years.

In the following variables, write the number of hours spent on this lab, the perceived difficulty, and the expected grade (you may delete the `raise` and the comments):

In [ ]:
# meta_eval manual_graded_answer 0

horas_gastas = None    # 1.5   - Float number with the number of hours spent 
dificuldade_lab = None # 0     - Float number from 0.0 to 10.0 (inclusive)
nota_esperada = None   # 10    - Float number from 0.0 to 10.0 (inclusive)

# WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
raise NotImplementedError()

Write below other comments or feedbacks about the lab. If you did not understand anything about the lab, please also comment here.

If you find any typo or bug in the lab, please comment below so we can fix it.

WRITE YOUR SOLUTION HERE! (do not change this first line):

**ATTENTION**

**ATTENTION**

**ATTENTION**

**ATTENTION**

**DISCURSIVE QUESTION**

WRITE YOUR ANSWER HERE (do not delete this cell so the ID is not lost)

**ATTENTION**

**ATTENTION**


**End of the lab!**